EXERCICE 1- ICO- DUFAURE DE LAJARTE - DURANTEAU

In [1]:
import numpy as np
M1=[6,3,10,14,5,9,7,11,2,3]
M2=[1,5,4,6,10,6,9,8,6,1]
M3=[5,8,1,3,6,10,12,9,6,7]
T=np.array([M1,M2,M3])

In [82]:
def min_liste_indice(M):
    if len(M) == 0:
        return "il n'y a pas de min car la liste est vide"
    else:
        imin = 0
        min_val = M[0]  # Renamed 'min' to 'min_val'
        for i in range(len(M)):
            if M[i] < min_val:
                imin = i
                min_val = M[i]
        return imin


In [130]:
import copy

def flow_shop_algorithm(task_times):
    # Création d'une copie profonde de la liste des temps de chaque tâche
    tasks = copy.deepcopy(task_times)
    
    # Liste pour stocker l'ordre d'exécution des tâches sur les trois machines
    execution_order = [[], [], []]  
    
    # Liste pour suivre l'indice de la tâche en cours et la date de fin de chaque tâche sur les trois machines
    task_indices = [[None, 0], [None, 0], [None, 0]]  

    # Fonction pour vérifier si une machine spécifique est occupée
    def is_machine_busy(machine_index):
        return task_indices[machine_index][1] != 0

    # Fonction pour trouver les tâches disponibles pour une machine spécifique à un temps donné
    def available_tasks(machine_index, current_time):
        available = []
        for j, task in enumerate(tasks):
            if task[machine_index] != 0 and task_indices[machine_index][1] <= current_time:
                if machine_index == 0 or (machine_index > 0 and task[machine_index - 1] == 0):
                    available.append(j)
        return available

    # Fonction pour trouver la tâche prioritaire parmi celles disponibles pour une machine spécifique
    def find_priority_task(machine_index, available_tasks):
        min_time = float('inf')
        priority_task_index = None
        for j in available_tasks:
            if tasks[j][machine_index] < min_time:
                min_time = tasks[j][machine_index]
                priority_task_index = j
        return priority_task_index

    # Initialisation du temps
    current_time = 0
    
    # Tant que chaque machine n'a pas exécuté 10 tâches
    while len(execution_order[0]) != 10 or len(execution_order[1]) != 10 or len(execution_order[2]) != 10:
        for machine_index in range(3):
            if not is_machine_busy(machine_index) and len(execution_order[machine_index]) < 10:
                available_tasks_list = available_tasks(machine_index, current_time)
                if available_tasks_list:
                    priority_task = find_priority_task(machine_index, available_tasks_list)
                    execution_order[machine_index].append(priority_task)
                    task_indices[machine_index][0] = priority_task
                    task_indices[machine_index][1] = tasks[priority_task][machine_index]
                    print(f"Au temps t = {current_time}, la tâche {priority_task} de durée {tasks[priority_task][machine_index]} débute sur la machine {machine_index + 1}.")
                    tasks[priority_task][machine_index] = 0
        current_time += 1
    
    print(execution_order)  # Afficher l'ordre d'exécution des tâches
    total_time = max(task_indices[2][1], task_indices[1][1], task_indices[0][1])  # Calcul du temps total d'exécution
    print(f"Temps total d'exécution : {total_time}")

Méthode par voisinage

In [ ]:
#calculer le temps total pour chaque permutation de tâches P, ie la fonction f(x) ici

def compute_total_time(T, P):
    total_time = [0] * len(T[0])
    for i in range(len(T)):
        for j in range(len(T[0])):
            if j==0 : 
                total_time[j] += T[i][P[i]] 
            else :
                for k in range(i+1):
                    max(total_time[j-1], sum(T[k][:P[i]+1])) 
    return max(total_time)

In [ ]:
def generate_neighbour(P):
    # On choisit deux indices aléatoires différents pour échanger les tâches
    idx1, idx2 = np.random.choice(len(P), 2, replace=False)
    # On échange les tâches aux deux indices choisis
    P[idx1], P[idx2] = P[idx2], P[idx1]
    
    return P

In [ ]:
max_iterations=1000

def algo_voisinage_descente(T, max_iterations):
    num_tasks = len(T[0])
    # Créez une solution initiale aléatoire
    initial_solution = list(range(len(T[0])))
    np.random.shuffle(initial_solution)
    
    # Calculez le temps total d'exécution pour la solution initiale
    best_solution = initial_solution
    best_time = compute_total_time(T, best_solution)
    
    # Effectuez les itérations de la méthode de descente
    for _ in range(max_iterations):
        # Générer une solution voisine
        neighbour_solution = generate_neighbour(best_solution.copy())
        
        # Calculez le temps total d'exécution pour la solution voisine
        neighbour_time = compute_total_time(T, neighbour_solution)
        
        # Vérifiez si la solution voisine est meilleure que la solution actuelle
        if neighbour_time < best_time:
            best_solution = neighbour_solution
            best_time = neighbour_time

    return best_solution, best_time

In [ ]:
algo_voisinage_descente(T,200)

In [ ]:
print(best_permutation)

len(T)

In [ ]:
P = [2, 0, 1, 4, 3, 5, 7, 6, 8, 9]

In [ ]:
compute_total_time(T, P)

Méthode Johnson-2 avec machine virtuelle M1+M2

In [ ]:
def johnson2_method(T):
    num_tasks = len(T[0])
    
    # Créez une liste d'indices de tâches
    indices = list(range(num_tasks))
    
    # Créer une machine virtuelle en combinant les temps de traitement de M1 et M2
    virtual_machine = [T[0][i] + T[1][i] for i in range(num_tasks)]

    # Trier les indices selon la machine virtuelle
    sorted_indices = sorted(indices, key=lambda x: virtual_machine[x])

    # Ordonner les tâches selon l'algorithme de Johnson
    U, V = [], []
    
    for i in range(num_tasks):
        if T[0][sorted_indices[i]] < T[1][sorted_indices[i]]:
            U.append(sorted_indices[i])
        else:
            V.append(sorted_indices[i])
    
    # Ordonner les tâches dans U et V
    LU = sorted(U, key=lambda x: T[0][x])
    LV = sorted(V, key=lambda x: T[1][x], reverse=True)
    
    # Concaténer les listes ordonnées
    L = LU + LV

    return L


In [ ]:
johnson_method(T)

# Algorithme Tabou

In [18]:
import numpy as np
import pandas as pd
import random as r

In [3]:
#on importe les donnée sur les clients et le depot
df=pd.read_excel("2_detail_table_customers.xls")
df_depot=pd.read_excel('4_detail_table_depots.xls')
#on extrait les informations qui nous intéressent
Columns=["CUSTOMER_NUMBER","CUSTOMER_LATITUDE","CUSTOMER_LONGITUDE"]
Data =( df[Columns]
       .rename(columns={"CUSTOMER_NUMBER":"number","CUSTOMER_LATITUDE":"x","CUSTOMER_LONGITUDE":"y"})
      )
depot = df_depot[['DEPOT_CODE','DEPOT_LATITUDE','DEPOT_LONGITUDE']].drop_duplicates()
depot = depot.rename(columns={"DEPOT_CODE":"number of client/depot","DEPOT_LATITUDE":"x","DEPOT_LONGITUDE":"y"})
Data.loc[len(Data)]=depot.iloc[0]

#définition d'une matrice U qui contient les distances entre les clienst/depot
U=np.zeros((131,131))
for i in range(131):
    for j in range(i+1,131):
        U[i,j]=np.sqrt((Data.iloc[i,1]-Data.iloc[j,1])**2+(Data.iloc[i,2]-Data.iloc[j,2])**2)
        U[j,i]=U[i,j]

In [16]:
nbmax = 100  # Nombre maximal d'itérations
f_limite = 0  # Borne inférieure de la fonction objectif
taille_max_T = 10  # Taille maximale de la liste taboue



In [61]:
def fonction_objectif( L,M=U):
    time=0
    V=30
    for i in range(len(L)-1):
        time+=U[L[i]-1,L[i+1]-1]/V
    return time


In [2]:
def generer_voisins(solution):
    voisins = []
    # Exemple : génération de voisins en inversant deux éléments de la solution
    for i in range(len(solution)):
        for j in range(i+1, len(solution)):
            voisin = solution[:]
            voisin[i], voisin[j] = voisin[j], voisin[i]
            voisins.append(voisin)
    return voisins

In [11]:
def fonction_aspiration(voisins,T,A):
    # Vérifier si la solution est taboue et satisfait l'aspiration
    sol=voisins[0][:]
    
    for solution in voisins[1:]:
        if (solution in T)  and (fonction_objectif(solution)<fonction_objectif(sol)):
            sol=solution[:]
    
    T.remove(sol)
    return sol

In [46]:
def initialiser_solution():
    L=[131]
    aux=[i for i in range(130)]
    r.shuffle(aux)
    L.extend(aux)
    L.append(131)
    return L

In [63]:
def tabou( nbmax):
    # Initialisation
    solution = initialiser_solution()
    meilleure_solution = solution[:]
    nb_iter = 0
    T = []  # Liste taboue initialement vide
    meilleure_iteration = 0
    f_limite = 50

    # Initialisation de la fonction d'aspiration
    A = fonction_objectif(meilleure_solution)

    # Processus itératif
    while (fonction_objectif(solution) > f_limite) and (nb_iter - meilleure_iteration < nbmax):
        nb_iter += 1
        voisins = generer_voisins(solution)
        meilleure_voisin = solution[:]
        for voisin in voisins:
            if (voisin not in T) or (fonction_objectif(voisin) <= A):
                meilleure_voisin = voisin[:]

        if meilleure_voisin == solution:
            # Si aucun voisin valide n'est trouvé, continuer avec la prochaine itération
            meilleure_voisin=fonction_aspiration(voisin,T,A)    
        

        # Mettre à jour la meilleure solution trouvée
        if fonction_objectif(meilleure_voisin) < fonction_objectif(meilleure_solution):
            meilleure_solution = meilleure_voisin[:]
            meilleure_iteration = nb_iter

        # Mettre à jour la liste taboue
        T.append(meilleure_voisin)
        if len(T) > taille_max_T:
            T.pop(0)  # Supprimer le plus ancien élément de T

        # Mettre à jour la fonction d'aspiration
        A = fonction_objectif(meilleure_solution)

        # Mettre à jour la solution courante
        solution = meilleure_voisin[:]

    return meilleure_solution


In [66]:
tabou(nbmax)

[131,
 78,
 38,
 47,
 70,
 114,
 60,
 73,
 8,
 126,
 58,
 35,
 92,
 31,
 36,
 117,
 110,
 116,
 13,
 2,
 81,
 83,
 120,
 123,
 125,
 127,
 86,
 100,
 111,
 77,
 107,
 14,
 54,
 44,
 50,
 89,
 4,
 12,
 67,
 69,
 91,
 61,
 88,
 40,
 74,
 82,
 90,
 7,
 29,
 109,
 118,
 33,
 10,
 76,
 115,
 112,
 25,
 104,
 56,
 65,
 48,
 80,
 15,
 20,
 1,
 87,
 16,
 26,
 94,
 98,
 52,
 121,
 129,
 124,
 68,
 9,
 84,
 24,
 99,
 85,
 42,
 128,
 34,
 17,
 59,
 39,
 55,
 71,
 95,
 113,
 32,
 11,
 22,
 23,
 6,
 105,
 18,
 27,
 45,
 49,
 63,
 41,
 72,
 102,
 37,
 5,
 0,
 93,
 30,
 119,
 62,
 53,
 3,
 103,
 43,
 66,
 97,
 64,
 106,
 101,
 19,
 75,
 46,
 122,
 79,
 21,
 51,
 28,
 108,
 57,
 96,
 131]

In [28]:
l=[1,2,3]
r.shuffle(l)
l

[2, 3, 1]